### 导入相关库

In [1]:
import os
import warnings
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = 'all'    # 默认为'last'

np.random.seed(2019)

### 导入数据&划分数据

In [2]:
data_path = './data/'
train_user_reply_data = pd.read_csv(data_path + 'train_user_reply_data.csv', encoding='utf-8')
train_search_data = pd.read_csv(data_path + 'train_search_data.csv', encoding='utf-8')
train_sales_data = pd.read_csv(data_path + 'train_sales_data.csv', encoding='utf-8')
evaluation_public = pd.read_csv(data_path + 'evaluation_public.csv', encoding='utf-8')

# 初赛数据
train_sales_data_preliminary = pd.read_csv('../chusai/data/train_sales_data.csv', encoding='utf-8')
added_model = set(train_sales_data['model']) - set(train_sales_data_preliminary['model'])
train_sales_data = train_sales_data[train_sales_data['model'].isin(added_model)]
train_user_reply_data = train_user_reply_data[train_user_reply_data['model'].isin(added_model)]
train_search_data = train_search_data[train_search_data['model'].isin(added_model)]
evaluation_public = evaluation_public[evaluation_public['model'].isin(added_model)]

train_sales_data = train_sales_data.sort_values(['regYear', 'regMonth', 'model', 'province'], ascending=True)

In [3]:
train_sales_data.sort_values(['regYear', 'regMonth', 'model', 'province'], ascending=True)

,province,adcode,model,bodyType,regYear,regMonth,salesVolume
38904,上海,310000,07a30393ec51530e,Sedan,2016,1,95
42072,云南,530000,07a30393ec51530e,Sedan,2016,1,192
43128,内蒙古,150000,07a30393ec51530e,Sedan,2016,1,202
34152,北京,110000,07a30393ec51530e,Sedan,2016,1,230
33096,四川,510000,07a30393ec51530e,Sedan,2016,1,467
...,...,...,...,...,...,...,...
37151,福建,350000,ffeb3a4cec627e8c,Sedan,2017,12,429
37679,辽宁,210000,ffeb3a4cec627e8c,Sedan,2017,12,287
39791,重庆,500000,ffeb3a4cec627e8c,Sedan,2017,12,339
36623,陕西,610000,ffeb3a4cec627e8c,Sedan,2017,12,337


### 新增22车型

In [4]:
cars = ['8cbac8dd8f0e89e2', '2509a94e62fd3e4b', 'a7128d308ae14d73', '32d3069d17aa47c2', '854982e5264fb53e', '07a30393ec51530e', '8527c11fccaa94e2', '9011c3cf495e3c7b', '28956642949e0e57', 'e8a33f68c15dcf7c', 'ffeb3a4cec627e8c', '9e967965ad4a6c07', 'a50a6db3d691e9ce', '0aa15031db420212', '283d331fdf05a655', '9a390098bf87b814', 'b135f55ced5c4c67', '42d4439853da7221', 'bc5bbd459c18fefd', 'b9aa8592ab2492da', '1181894ae2811540', '5f727f32393ade77']
provinces = ['浙江', '福建', '四川', '陕西', '安徽', '湖南', '广东', '云南', '上海', '山东', '湖北', '黑龙江', '江苏', '广西', '内蒙古', '辽宁', '北京', '重庆', '河北', '山西', '江西', '河南']

### 全量车型

In [5]:
# cars = ['2d0d2c3403909fdb', 'c6cd4e0e073f5ac2', 'ef76a85c4b39f693', 'f270f6a489c6a9d7', '1181894ae2811540', 'a7128d308ae14d73', '7023efdab9cedc03', '9a390098bf87b814', 'b4be3a4917289c82', 'cc21c7e91a3b5a0c', '4f79773e600518a6', 'c6833cb891626c17', '8c915fe4632fb9fa', 'da457d15788fe8ee', '28956642949e0e57', '28e29f2c03dcd84c', 'a432c483b5beb856', 'cd5841d44fd7625e', 'af6f4f548684e14d', '4a103c30d593fbbe', 'a28bb927b6fcb33c', '8cbac8dd8f0e89e2', '3d7554f1f56dd664', '17bc272c93f19d56', '5d7fb682edd0f937', '7aab7fca2470987e', 'a207df29ec9583f0', '7cf283430b3b5e38', 'bb9fbec9a2833839', 'e8a33f68c15dcf7c', '5b1c11c3efed5312', '02aab221aabc03b9', 'c06a2a387c0ee510', '3e21824be728cbec', 'd0f245b8781e3631', '61e73e32ad101892', '7a7885e2d7c00bcf', 'f5d69960089c3614', '2a2ab41f8f6ff1cb', '3c974920a76ac9c1', '0aa15031db420212', '854982e5264fb53e', 'a50a6db3d691e9ce', '2509a94e62fd3e4b', '04e66e578f653ab9', '0797526c057dcf5b', 'b135f55ced5c4c67', '6858d6dfe680bdf7', '9c1c7ee8ebdda299', 'b9aa8592ab2492da', '42d4439853da7221', 'd4efbebb087fd03f', 'ea489c253676aafc', '6155b214590c66e6', '07a30393ec51530e', '63065128401bb3ff', '7245e0ee27b195cd', '32d3069d17aa47c2', 'f8a6975573af1b33', '212083a9246d2fd3', '37aa9169b575ef79', 'fc32b1a017b34efe', 'a9a43d1a7ecbe75d', '8527c11fccaa94e2', 'bc5bbd459c18fefd', '12f8b7e14947c34d', '54fc07138d70374c', 'b25c4e2e3856af22', '9e967965ad4a6c07', '936168bd4850913d', 'dff803b4024d261d', 'fde95ea242abd896', '5f727f32393ade77', '9011c3cf495e3c7b', '283d331fdf05a655', '06880909932890ca', '17363f08d683d52b', 'ffeb3a4cec627e8c', '346393c2c6305fb1', '79de4e4b24c35b04', 'feabbf46658382b9', '97f15de12cfabbd5']
# provinces = ['浙江', '福建', '四川', '陕西', '安徽', '湖南', '广东', '云南', '上海', '山东', '湖北', '黑龙江', '江苏', '广西', '内蒙古', '辽宁', '北京', '重庆', '河北', '山西', '江西', '河南']

### 自定义评价指标

In [6]:
from sklearn.metrics import mean_squared_error

def metrics(y_true, y_pred, model):
    data = pd.DataFrame({'model': model, 'salesVolume': y_true, 'label': y_pred})
    data['label'] = data['label'].map(lambda index: -index if index < 0 else index)
    res, count = 0, 0
    for index, cars in data.groupby('model'):
        a = np.array(cars['salesVolume'])
        b = np.array(cars['label'])
        temp = np.sqrt(np.sum((a - b) ** 2) / len(a)) / np.mean(a)
        res += temp
        count += 1
        print(temp)
    return 1 - (res / count)

### 获取训练集/测试集索引

In [7]:
def get_train_feature(windows_size, before):
    features = pd.DataFrame()
    for car in cars:
        for province in provinces:
            car_province_part = train_sales_data[(train_sales_data['model'] == car) & (train_sales_data['province'] == province)]
            car_province_part['label'] = car_province_part['salesVolume'].shift(-windows_size)
            car_province_part = car_province_part[before:24 - windows_size]
            features = pd.concat([features, car_province_part], axis=0)
    features.index = range(len(features))
    return features

def get_test_feature(windows_size, before):
    features = pd.DataFrame()
    for car in cars:
        for province in provinces:
            car_province_part = train_sales_data[(train_sales_data['model'] == car) & (train_sales_data['province'] == province)]
            car_province_part['label'] = car_province_part['salesVolume'].shift(-windows_size)
            car_province_part = car_province_part[-1:]
            features = pd.concat([features, car_province_part], axis=0)
    features.index = range(len(features))
    return features

### 特征提取

In [8]:
def get_basic_feature(windows_size, before, data_set_name):
    features = pd.DataFrame()
    for car in cars:
        for province in provinces:
            car_province_part = train_sales_data[(train_sales_data['model'] == car) & (train_sales_data['province'] == province)].copy()
            car_province_part['salesVolume'] = car_province_part['salesVolume'].apply(lambda index: np.log(index))   ###

            car_province_part['is_pring_festival'] = [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
            car_province_part['distance_spring_festival'] = [1, 0, 1, 2, 3, 4, 5, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 6, 5, 4, 3, 2]

            # 一阶差分
            for index in range(1, before, 1):
                car_province_part['salesVolume_' + str(index)] = car_province_part['salesVolume'].shift(index)
                car_province_part['salesVolume_diff_' + str(index)] = car_province_part['salesVolume'].diff(index)
                car_province_part['salesVolume_qoq_' + str(index)] = car_province_part['salesVolume'] / car_province_part['salesVolume_' + str(index)]

            # 二阶差分
            for index in range(1, before - 1, 1):   
                car_province_part['salesVolume_diff2_{}'.format(str(index))] = car_province_part['salesVolume_diff_' + str(index)].diff(1)

            # 数据集划分
            if data_set_name == 'train':
                car_province_part = car_province_part[before : 24 - windows_size]
            else:
                car_province_part = car_province_part[-1:]

            features = pd.concat([features, car_province_part], axis=0, ignore_index=True)

    print(features)
    return features

### Begain

In [9]:
test_prob_collection = pd.DataFrame()

### Model-LightGBM - 一月

In [10]:
size, pre = 1, 10  # 4
train_feature  = get_train_feature(size, pre)
test_feature  = get_test_feature(size, pre)

cols = ['province', 'adcode', 'model', 'regYear', 'regMonth', 'bodyType']   #  , 'salesVolume'

categorial_name = [0, 1, 2, 3, 4, 7, 8]
# categorial_name = [0, 1, 2, 3, 4, 7, 8, 9, 10, 11]
drop_cols = ['salesVolume']  # 'popularity'

temp_train = get_basic_feature(size, pre, 'train')
train_feature = train_feature.drop(drop_cols, axis=1).merge(temp_train, on=cols, how='left')
train_feature

temp_test = get_basic_feature(size, pre, 'test')
test_feature = test_feature.drop(drop_cols, axis=1).merge(temp_test, on=cols, how='left')
test_feature
train_feature.isnull().sum()
test_feature

submit = test_feature[['province', 'adcode', 'model']]
submit['regYear'] = 2018
submit['regMonth'] = 1
###############################

test_index = list(train_feature[(train_feature['regYear'] == 2017) & (train_feature['regMonth'] == 11)].index)

def drop_duplicate(n):
    return n not in test_index

train_index = list(filter(drop_duplicate, list(range(len(train_feature)))))

train_model = train_feature['model'].values[train_index]   # model
val_model = train_feature['model'].values[test_index]

model_set = dict()
for index in range(len(cars)):
    model_set[cars[index]] = index
train_feature['bodyType'] = train_feature['bodyType'].map({'Hatchback': 0, 'MPV': 1, 'SUV': 2, 'Sedan': 3})
train_feature['model'] = train_feature['model'].map(model_set)
test_feature['bodyType'] = test_feature['bodyType'].map({'Hatchback': 0, 'MPV': 1, 'SUV': 2, 'Sedan': 3})
test_feature['model'] = test_feature['model'].map(model_set)

train_label = train_feature[['label']]
train_feature.drop(['province', 'label'], axis=1, inplace=True)
test_feature.drop(['province', 'label'], axis=1, inplace=True)

train_label['log'] = train_label['label'].apply(lambda index: np.log2(index) + 1)
x_train = train_feature.values[train_index]
y_train = train_label['log'].values[train_index]
x_test = train_feature.values[test_index]
y_test = train_label['log'].values[test_index]

     province  adcode             model   bodyType  regYear  regMonth  \
0          浙江  330000  8cbac8dd8f0e89e2        SUV     2016        11   
1          浙江  330000  8cbac8dd8f0e89e2        SUV     2016        12   
2          浙江  330000  8cbac8dd8f0e89e2        SUV     2017         1   
3          浙江  330000  8cbac8dd8f0e89e2        SUV     2017         2   
4          浙江  330000  8cbac8dd8f0e89e2        SUV     2017         3   
...       ...     ...               ...        ...      ...       ...   
6287       河南  410000  5f727f32393ade77  Hatchback     2017         7   
6288       河南  410000  5f727f32393ade77  Hatchback     2017         8   
6289       河南  410000  5f727f32393ade77  Hatchback     2017         9   
6290       河南  410000  5f727f32393ade77  Hatchback     2017        10   
6291       河南  410000  5f727f32393ade77  Hatchback     2017        11   

      salesVolume  is_pring_festival  distance_spring_festival  salesVolume_1  \
0        4.787492                  0      

,province,adcode,model,bodyType,regYear,regMonth,label,salesVolume,is_pring_festival,distance_spring_festival,...,salesVolume_diff_9,salesVolume_qoq_9,salesVolume_diff2_1,salesVolume_diff2_2,salesVolume_diff2_3,salesVolume_diff2_4,salesVolume_diff2_5,salesVolume_diff2_6,salesVolume_diff2_7,salesVolume_diff2_8
0,浙江,330000,8cbac8dd8f0e89e2,SUV,2016,11,193.0,4.787492,0,2,...,0.582799,1.138607,-0.253130,-0.112857,-0.362171,0.000000,0.107312,-0.187599,0.109815,-0.465016
1,浙江,330000,8cbac8dd8f0e89e2,SUV,2016,12,93.0,5.262690,0,1,...,0.657520,1.142779,0.539737,0.286607,0.426880,0.177566,0.539737,0.647049,0.352138,0.649552
2,浙江,330000,8cbac8dd8f0e89e2,SUV,2017,1,59.0,4.532599,1,0,...,0.101783,1.022972,-1.205289,-0.665552,-0.918682,-0.778409,-1.027723,-0.665552,-0.558240,-0.853151
3,浙江,330000,8cbac8dd8f0e89e2,SUV,2017,2,72.0,4.077537,0,1,...,-0.476339,0.895399,0.275029,-0.930260,-0.390524,-0.643653,-0.503381,-0.752694,-0.390524,-0.283212
4,浙江,330000,8cbac8dd8f0e89e2,SUV,2017,3,49.0,4.276666,0,2,...,-0.105361,0.975956,0.654191,0.929219,-0.276070,0.263667,0.010538,0.150810,-0.098504,0.263667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6287,河南,410000,5f727f32393ade77,Hatchback,2017,7,323.0,6.023448,0,6,...,0.129045,1.021893,-0.068318,0.166457,0.013724,-0.327213,-0.306700,0.851552,-0.259674,-0.155182
6288,河南,410000,5f727f32393ade77,Hatchback,2017,8,370.0,5.777652,0,6,...,-0.276787,0.954284,-0.250650,-0.318968,-0.084193,-0.236926,-0.577862,-0.557350,0.600902,-0.510324
6289,河南,410000,5f727f32393ade77,Hatchback,2017,9,309.0,5.913503,0,5,...,-0.405465,0.935834,0.381646,0.130996,0.062678,0.297453,0.144720,-0.196216,-0.175704,0.982548
6290,河南,410000,5f727f32393ade77,Hatchback,2017,10,367.0,5.733341,0,4,...,0.261071,1.047708,-0.316012,0.065634,-0.185016,-0.253334,-0.018559,-0.171292,-0.512229,-0.491716


    province  adcode             model   bodyType  regYear  regMonth  \
0         浙江  330000  8cbac8dd8f0e89e2        SUV     2017        12   
1         福建  350000  8cbac8dd8f0e89e2        SUV     2017        12   
2         四川  510000  8cbac8dd8f0e89e2        SUV     2017        12   
3         陕西  610000  8cbac8dd8f0e89e2        SUV     2017        12   
4         安徽  340000  8cbac8dd8f0e89e2        SUV     2017        12   
..       ...     ...               ...        ...      ...       ...   
479       重庆  500000  5f727f32393ade77  Hatchback     2017        12   
480       河北  130000  5f727f32393ade77  Hatchback     2017        12   
481       山西  140000  5f727f32393ade77  Hatchback     2017        12   
482       江西  360000  5f727f32393ade77  Hatchback     2017        12   
483       河南  410000  5f727f32393ade77  Hatchback     2017        12   

     salesVolume  is_pring_festival  distance_spring_festival  salesVolume_1  \
0       4.248495                  0                    

,province,adcode,model,bodyType,regYear,regMonth,label,salesVolume,is_pring_festival,distance_spring_festival,...,salesVolume_diff_9,salesVolume_qoq_9,salesVolume_diff2_1,salesVolume_diff2_2,salesVolume_diff2_3,salesVolume_diff2_4,salesVolume_diff2_5,salesVolume_diff2_6,salesVolume_diff2_7,salesVolume_diff2_8
0,浙江,330000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,4.248495,0,2,...,-0.028171,0.993413,1.554908,0.902801,0.970669,0.473205,0.707263,1.092711,0.457833,1.199354
1,福建,350000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,4.204693,0,2,...,-0.522695,0.889433,-0.677515,-0.568700,-0.589720,-0.292964,-0.999731,-0.090032,-1.202824,0.004103
2,四川,510000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,4.941642,0,2,...,0.259511,1.055426,0.596242,0.399192,0.292823,0.546400,-0.052922,0.227170,0.143788,0.890180
3,陕西,610000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,5.438079,0,2,...,1.377636,1.339282,1.560052,0.867843,0.651712,1.219107,0.433917,0.991002,1.407248,0.556846
4,安徽,340000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,3.258097,0,2,...,0.367725,1.127224,0.238411,-0.277632,-0.820981,0.241162,0.133531,0.810930,-0.325422,-0.367725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,重庆,500000,5f727f32393ade77,Hatchback,2017,12,NaN,4.852030,0,2,...,0.508225,1.117000,0.251104,0.551030,0.061252,0.401701,0.479454,0.133962,0.255207,0.330242
480,河北,130000,5f727f32393ade77,Hatchback,2017,12,NaN,6.991177,0,2,...,0.411926,1.062610,0.413524,0.632738,0.632097,0.575226,0.533479,0.673928,0.388894,0.310814
481,山西,140000,5f727f32393ade77,Hatchback,2017,12,NaN,5.863631,0,2,...,0.404046,1.074007,0.589077,0.601244,0.594485,0.524223,0.597950,0.419903,0.454634,0.768264
482,江西,360000,5f727f32393ade77,Hatchback,2017,12,NaN,5.455321,0,2,...,0.208297,1.039698,0.100234,0.312397,0.419217,0.119323,0.203010,0.381650,0.239431,0.528983


province                    0
adcode                      0
model                       0
bodyType                    0
regYear                     0
regMonth                    0
label                       0
salesVolume                 0
is_pring_festival           0
distance_spring_festival    0
salesVolume_1               0
salesVolume_diff_1          0
salesVolume_qoq_1           9
salesVolume_2               0
salesVolume_diff_2          0
salesVolume_qoq_2           7
salesVolume_3               0
salesVolume_diff_3          0
salesVolume_qoq_3           7
salesVolume_4               0
salesVolume_diff_4          0
salesVolume_qoq_4           7
salesVolume_5               0
salesVolume_diff_5          0
salesVolume_qoq_5           9
salesVolume_6               0
salesVolume_diff_6          0
salesVolume_qoq_6           7
salesVolume_7               0
salesVolume_diff_7          0
salesVolume_qoq_7           7
salesVolume_8               0
salesVolume_diff_8          0
salesVolum

,province,adcode,model,bodyType,regYear,regMonth,label,salesVolume,is_pring_festival,distance_spring_festival,...,salesVolume_diff_9,salesVolume_qoq_9,salesVolume_diff2_1,salesVolume_diff2_2,salesVolume_diff2_3,salesVolume_diff2_4,salesVolume_diff2_5,salesVolume_diff2_6,salesVolume_diff2_7,salesVolume_diff2_8
0,浙江,330000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,4.248495,0,2,...,-0.028171,0.993413,1.554908,0.902801,0.970669,0.473205,0.707263,1.092711,0.457833,1.199354
1,福建,350000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,4.204693,0,2,...,-0.522695,0.889433,-0.677515,-0.568700,-0.589720,-0.292964,-0.999731,-0.090032,-1.202824,0.004103
2,四川,510000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,4.941642,0,2,...,0.259511,1.055426,0.596242,0.399192,0.292823,0.546400,-0.052922,0.227170,0.143788,0.890180
3,陕西,610000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,5.438079,0,2,...,1.377636,1.339282,1.560052,0.867843,0.651712,1.219107,0.433917,0.991002,1.407248,0.556846
4,安徽,340000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,3.258097,0,2,...,0.367725,1.127224,0.238411,-0.277632,-0.820981,0.241162,0.133531,0.810930,-0.325422,-0.367725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,重庆,500000,5f727f32393ade77,Hatchback,2017,12,NaN,4.852030,0,2,...,0.508225,1.117000,0.251104,0.551030,0.061252,0.401701,0.479454,0.133962,0.255207,0.330242
480,河北,130000,5f727f32393ade77,Hatchback,2017,12,NaN,6.991177,0,2,...,0.411926,1.062610,0.413524,0.632738,0.632097,0.575226,0.533479,0.673928,0.388894,0.310814
481,山西,140000,5f727f32393ade77,Hatchback,2017,12,NaN,5.863631,0,2,...,0.404046,1.074007,0.589077,0.601244,0.594485,0.524223,0.597950,0.419903,0.454634,0.768264
482,江西,360000,5f727f32393ade77,Hatchback,2017,12,NaN,5.455321,0,2,...,0.208297,1.039698,0.100234,0.312397,0.419217,0.119323,0.203010,0.381650,0.239431,0.528983


In [11]:
# LightGBM model
params = {
      'boosting_type': 'gbdt',
      'objective': 'rmse',
      'metric': ['rmse'],   # 'l2', 'binary_logloss',
      'learning_rate': 0.03,
      'num_leaves': 2 ** 5 - 1,    # 2 ** 5 - 1
      # 'min_child_samples': 100,
      'max_depth': 6,    # 6
      'subsample': 0.8,   # 0.8
      'subsample_freq': 5,
      'colsample_bytree': 0.8,
      'seed': 2019,
      'nthread': -1,
      'verbose': 1,
}

lgb_train = lgb.Dataset(x_train, y_train.ravel())
lgb_eval = lgb.Dataset(x_test, y_test.ravel(), reference=lgb_train)
# categorial_name = ['adcode', 'model', 'bodyType', 'regYear', 'regMonth']

# num_boost_round: 5000   early_stopping_rounds:100
module = lgb.train(params, lgb_train, num_boost_round=5000, valid_sets=lgb_eval, early_stopping_rounds=100, categorical_feature=categorial_name)

# feature importance
importance = module.feature_importance()
print(importance)

val = module.predict(x_test, num_iteration=module.best_iteration)
val = 2 ** (val - 1)
y_true = 2 ** (y_test.reshape(1, -1)[0] - 1)
nrmse = metrics(y_true, val, val_model.reshape(1, -1)[0])

iters = module.best_iteration + 100
train_all = np.vstack((x_train, x_test))
label_all = np.hstack((y_train, y_test))
lgb_data = lgb.Dataset(train_all, label_all.ravel())
model = lgb.train(params, lgb_data, num_boost_round=iters, categorical_feature=categorial_name)

predict = model.predict(test_feature)
predict = 2 ** (predict - 1)
print(predict)

print('model train over, rmse:', nrmse)   
submit['forecastVolum'] = predict
test_prob_collection = pd.concat([test_prob_collection, submit], axis=0, ignore_index=True)
print(train_feature.shape)

[1]	valid_0's rmse: 1.73867
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 1.69205
[3]	valid_0's rmse: 1.64696
[4]	valid_0's rmse: 1.60312
[5]	valid_0's rmse: 1.56041
[6]	valid_0's rmse: 1.52025
[7]	valid_0's rmse: 1.4803
[8]	valid_0's rmse: 1.44383
[9]	valid_0's rmse: 1.40649
[10]	valid_0's rmse: 1.37205
[11]	valid_0's rmse: 1.33825
[12]	valid_0's rmse: 1.3054
[13]	valid_0's rmse: 1.27307
[14]	valid_0's rmse: 1.24195
[15]	valid_0's rmse: 1.21277
[16]	valid_0's rmse: 1.18447
[17]	valid_0's rmse: 1.15744
[18]	valid_0's rmse: 1.13063
[19]	valid_0's rmse: 1.10395
[20]	valid_0's rmse: 1.07846
[21]	valid_0's rmse: 1.05342
[22]	valid_0's rmse: 1.0297
[23]	valid_0's rmse: 1.00611
[24]	valid_0's rmse: 0.984781
[25]	valid_0's rmse: 0.962914
[26]	valid_0's rmse: 0.942333
[27]	valid_0's rmse: 0.922164
[28]	valid_0's rmse: 0.903102
[29]	valid_0's rmse: 0.884852
[30]	valid_0's rmse: 0.867077
[31]	valid_0's rmse: 0.850536
[32]	valid_0's rmse: 0.833685
[33]	valid_0

[327]	valid_0's rmse: 0.398214
[328]	valid_0's rmse: 0.398295
[329]	valid_0's rmse: 0.398252
[330]	valid_0's rmse: 0.398335
[331]	valid_0's rmse: 0.398342
[332]	valid_0's rmse: 0.398385
[333]	valid_0's rmse: 0.398429
[334]	valid_0's rmse: 0.398411
[335]	valid_0's rmse: 0.398369
[336]	valid_0's rmse: 0.398402
Early stopping, best iteration is:
[236]	valid_0's rmse: 0.397278
[364 593 117 174 676 720 174  50   6 113 117 334 147 123 106  98  79  88
  76  80  81  62  74 118  93  90 151 106  76 126 140  89  74 103  83 127
 150  91 154  93  97  99 154]
0.18637975795460596
0.5050241221900799
0.19427866617730508
0.335712366072519
0.24681380559824076
0.33707214511694245
0.1500665221792039
0.27890715479807876
0.4925010698711372
0.217887063935413
0.11459728691786783
0.5523330752007508
0.3057165892469134
0.16280323989359616
0.4296562787624702
0.3548800611277685
0.2729926388001687
0.3213129339685741
0.23890658002244602
0.40067961755508685
0.2003408138403262
0.49694460571721694
[5.71501080e+01 6.5454

```
model train over, rmse: 0.6958471432146117
```

### Model-LightGBM - 二月

In [12]:
size, pre = 2, 9  # 4
train_feature  = get_train_feature(size, pre)
test_feature  = get_test_feature(size, pre)

cols = ['province', 'adcode', 'model', 'regYear', 'regMonth', 'bodyType']   #  , 'salesVolume'

temp_train = get_basic_feature(size, pre, 'train')
train_feature = train_feature.drop(drop_cols, axis=1).merge(temp_train, on=cols, how='left')
train_feature

temp_test = get_basic_feature(size, pre, 'test')
test_feature = test_feature.drop(drop_cols, axis=1).merge(temp_test, on=cols, how='left')
test_feature
train_feature.isnull().sum()
test_feature

submit = test_feature[['province', 'adcode', 'model']]
submit['regYear'] = 2018
submit['regMonth'] = 2
######################################

test_index = list(train_feature[(train_feature['regYear'] == 2017) & (train_feature['regMonth'] == 10)].index)

def drop_duplicate(n):
    return n not in test_index

train_index = list(filter(drop_duplicate, list(range(len(train_feature)))))

train_model = train_feature['model'].values[train_index]   # model
val_model = train_feature['model'].values[test_index]

model_set = dict()
for index in range(len(cars)):
    model_set[cars[index]] = index
train_feature['bodyType'] = train_feature['bodyType'].map({'Hatchback': 0, 'MPV': 1, 'SUV': 2, 'Sedan': 3})
train_feature['model'] = train_feature['model'].map(model_set)
test_feature['bodyType'] = test_feature['bodyType'].map({'Hatchback': 0, 'MPV': 1, 'SUV': 2, 'Sedan': 3})
test_feature['model'] = test_feature['model'].map(model_set)

train_label = train_feature[['label']]
train_feature.drop(['province', 'label'], axis=1, inplace=True)
test_feature.drop(['province', 'label'], axis=1, inplace=True)

train_label['log'] = train_label['label'].apply(lambda index: np.log2(index) + 1)
x_train = train_feature.values[train_index]
y_train = train_label['log'].values[train_index]
x_test = train_feature.values[test_index]
y_test = train_label['log'].values[test_index]

     province  adcode             model   bodyType  regYear  regMonth  \
0          浙江  330000  8cbac8dd8f0e89e2        SUV     2016        10   
1          浙江  330000  8cbac8dd8f0e89e2        SUV     2016        11   
2          浙江  330000  8cbac8dd8f0e89e2        SUV     2016        12   
3          浙江  330000  8cbac8dd8f0e89e2        SUV     2017         1   
4          浙江  330000  8cbac8dd8f0e89e2        SUV     2017         2   
...       ...     ...               ...        ...      ...       ...   
6287       河南  410000  5f727f32393ade77  Hatchback     2017         6   
6288       河南  410000  5f727f32393ade77  Hatchback     2017         7   
6289       河南  410000  5f727f32393ade77  Hatchback     2017         8   
6290       河南  410000  5f727f32393ade77  Hatchback     2017         9   
6291       河南  410000  5f727f32393ade77  Hatchback     2017        10   

      salesVolume  is_pring_festival  distance_spring_festival  salesVolume_1  \
0        4.852030                  0      

,province,adcode,model,bodyType,regYear,regMonth,label,salesVolume,is_pring_festival,distance_spring_festival,...,salesVolume_8,salesVolume_diff_8,salesVolume_qoq_8,salesVolume_diff2_1,salesVolume_diff2_2,salesVolume_diff2_3,salesVolume_diff2_4,salesVolume_diff2_5,salesVolume_diff2_6,salesVolume_diff2_7
0,浙江,330000,8cbac8dd8f0e89e2,SUV,2016,10,193.0,4.852030,0,3,...,4.204693,0.647338,1.153956,0.140273,-0.109041,0.253130,0.360441,0.065531,0.362945,-0.211886
1,浙江,330000,8cbac8dd8f0e89e2,SUV,2016,11,93.0,4.787492,0,2,...,4.605170,0.182322,1.039591,-0.253130,-0.112857,-0.362171,0.000000,0.107312,-0.187599,0.109815
2,浙江,330000,8cbac8dd8f0e89e2,SUV,2016,12,59.0,5.262690,0,1,...,4.430817,0.831873,1.187747,0.539737,0.286607,0.426880,0.177566,0.539737,0.647049,0.352138
3,浙江,330000,8cbac8dd8f0e89e2,SUV,2017,1,72.0,4.532599,1,0,...,4.553877,-0.021277,0.995328,-1.205289,-0.665552,-0.918682,-0.778409,-1.027723,-0.665552,-0.558240
4,浙江,330000,8cbac8dd8f0e89e2,SUV,2017,2,49.0,4.077537,0,1,...,4.382027,-0.304489,0.930514,0.275029,-0.930260,-0.390524,-0.643653,-0.503381,-0.752694,-0.390524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6287,河南,410000,5f727f32393ade77,Hatchback,2017,6,323.0,6.018593,0,5,...,5.894403,0.124190,1.021069,0.234775,0.082042,-0.258894,-0.238382,0.919870,-0.191356,-0.086864
6288,河南,410000,5f727f32393ade77,Hatchback,2017,7,370.0,6.023448,0,6,...,6.054439,-0.030992,0.994881,-0.068318,0.166457,0.013724,-0.327213,-0.306700,0.851552,-0.259674
6289,河南,410000,5f727f32393ade77,Hatchback,2017,8,309.0,5.777652,0,6,...,6.318968,-0.541316,0.914335,-0.250650,-0.318968,-0.084193,-0.236926,-0.577862,-0.557350,0.600902
6290,河南,410000,5f727f32393ade77,Hatchback,2017,9,367.0,5.913503,0,5,...,5.472271,0.441232,1.080631,0.381646,0.130996,0.062678,0.297453,0.144720,-0.196216,-0.175704


    province  adcode             model   bodyType  regYear  regMonth  \
0         浙江  330000  8cbac8dd8f0e89e2        SUV     2017        12   
1         福建  350000  8cbac8dd8f0e89e2        SUV     2017        12   
2         四川  510000  8cbac8dd8f0e89e2        SUV     2017        12   
3         陕西  610000  8cbac8dd8f0e89e2        SUV     2017        12   
4         安徽  340000  8cbac8dd8f0e89e2        SUV     2017        12   
..       ...     ...               ...        ...      ...       ...   
479       重庆  500000  5f727f32393ade77  Hatchback     2017        12   
480       河北  130000  5f727f32393ade77  Hatchback     2017        12   
481       山西  140000  5f727f32393ade77  Hatchback     2017        12   
482       江西  360000  5f727f32393ade77  Hatchback     2017        12   
483       河南  410000  5f727f32393ade77  Hatchback     2017        12   

     salesVolume  is_pring_festival  distance_spring_festival  salesVolume_1  \
0       4.248495                  0                    

,province,adcode,model,bodyType,regYear,regMonth,label,salesVolume,is_pring_festival,distance_spring_festival,...,salesVolume_8,salesVolume_diff_8,salesVolume_qoq_8,salesVolume_diff2_1,salesVolume_diff2_2,salesVolume_diff2_3,salesVolume_diff2_4,salesVolume_diff2_5,salesVolume_diff2_6,salesVolume_diff2_7
0,浙江,330000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,4.248495,0,2,...,3.891820,0.356675,1.091647,1.554908,0.902801,0.970669,0.473205,0.707263,1.092711,0.457833
1,福建,350000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,4.204693,0,2,...,4.174387,0.030305,1.007260,-0.677515,-0.568700,-0.589720,-0.292964,-0.999731,-0.090032,-1.202824
2,四川,510000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,4.941642,0,2,...,4.158883,0.782759,1.188214,0.596242,0.399192,0.292823,0.546400,-0.052922,0.227170,0.143788
3,陕西,610000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,5.438079,0,2,...,4.510860,0.927220,1.205553,1.560052,0.867843,0.651712,1.219107,0.433917,0.991002,1.407248
4,安徽,340000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,3.258097,0,2,...,3.258097,0.000000,1.000000,0.238411,-0.277632,-0.820981,0.241162,0.133531,0.810930,-0.325422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,重庆,500000,5f727f32393ade77,Hatchback,2017,12,NaN,4.852030,0,2,...,4.343805,0.508225,1.117000,0.251104,0.551030,0.061252,0.401701,0.479454,0.133962,0.255207
480,河北,130000,5f727f32393ade77,Hatchback,2017,12,NaN,6.991177,0,2,...,6.776507,0.214670,1.031679,0.413524,0.632738,0.632097,0.575226,0.533479,0.673928,0.388894
481,山西,140000,5f727f32393ade77,Hatchback,2017,12,NaN,5.863631,0,2,...,5.241747,0.621884,1.118641,0.589077,0.601244,0.594485,0.524223,0.597950,0.419903,0.454634
482,江西,360000,5f727f32393ade77,Hatchback,2017,12,NaN,5.455321,0,2,...,4.997212,0.458109,1.091673,0.100234,0.312397,0.419217,0.119323,0.203010,0.381650,0.239431


province                    0
adcode                      0
model                       0
bodyType                    0
regYear                     0
regMonth                    0
label                       0
salesVolume                 0
is_pring_festival           0
distance_spring_festival    0
salesVolume_1               0
salesVolume_diff_1          0
salesVolume_qoq_1           9
salesVolume_2               0
salesVolume_diff_2          0
salesVolume_qoq_2           8
salesVolume_3               0
salesVolume_diff_3          0
salesVolume_qoq_3           8
salesVolume_4               0
salesVolume_diff_4          0
salesVolume_qoq_4           8
salesVolume_5               0
salesVolume_diff_5          0
salesVolume_qoq_5           9
salesVolume_6               0
salesVolume_diff_6          0
salesVolume_qoq_6           7
salesVolume_7               0
salesVolume_diff_7          0
salesVolume_qoq_7           7
salesVolume_8               0
salesVolume_diff_8          0
salesVolum

,province,adcode,model,bodyType,regYear,regMonth,label,salesVolume,is_pring_festival,distance_spring_festival,...,salesVolume_8,salesVolume_diff_8,salesVolume_qoq_8,salesVolume_diff2_1,salesVolume_diff2_2,salesVolume_diff2_3,salesVolume_diff2_4,salesVolume_diff2_5,salesVolume_diff2_6,salesVolume_diff2_7
0,浙江,330000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,4.248495,0,2,...,3.891820,0.356675,1.091647,1.554908,0.902801,0.970669,0.473205,0.707263,1.092711,0.457833
1,福建,350000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,4.204693,0,2,...,4.174387,0.030305,1.007260,-0.677515,-0.568700,-0.589720,-0.292964,-0.999731,-0.090032,-1.202824
2,四川,510000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,4.941642,0,2,...,4.158883,0.782759,1.188214,0.596242,0.399192,0.292823,0.546400,-0.052922,0.227170,0.143788
3,陕西,610000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,5.438079,0,2,...,4.510860,0.927220,1.205553,1.560052,0.867843,0.651712,1.219107,0.433917,0.991002,1.407248
4,安徽,340000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,3.258097,0,2,...,3.258097,0.000000,1.000000,0.238411,-0.277632,-0.820981,0.241162,0.133531,0.810930,-0.325422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,重庆,500000,5f727f32393ade77,Hatchback,2017,12,NaN,4.852030,0,2,...,4.343805,0.508225,1.117000,0.251104,0.551030,0.061252,0.401701,0.479454,0.133962,0.255207
480,河北,130000,5f727f32393ade77,Hatchback,2017,12,NaN,6.991177,0,2,...,6.776507,0.214670,1.031679,0.413524,0.632738,0.632097,0.575226,0.533479,0.673928,0.388894
481,山西,140000,5f727f32393ade77,Hatchback,2017,12,NaN,5.863631,0,2,...,5.241747,0.621884,1.118641,0.589077,0.601244,0.594485,0.524223,0.597950,0.419903,0.454634
482,江西,360000,5f727f32393ade77,Hatchback,2017,12,NaN,5.455321,0,2,...,4.997212,0.458109,1.091673,0.100234,0.312397,0.419217,0.119323,0.203010,0.381650,0.239431


In [13]:
# LightGBM model
params = {
      'boosting_type': 'gbdt',
      'objective': 'rmse',
      'metric': ['rmse'],   # 'l2', 'binary_logloss',
      'learning_rate': 0.03,
      'num_leaves': 2 ** 5 - 1,
      # 'min_child_samples': 100,
      'max_depth': 6,
      'subsample': 0.8,
      'subsample_freq': 5,
      'colsample_bytree': 0.8,
      'seed': 2019,
      'nthread': -1,
      'verbose': 1,
}

lgb_train = lgb.Dataset(x_train, y_train.ravel())
lgb_eval = lgb.Dataset(x_test, y_test.ravel(), reference=lgb_train)
# categorial_name = ['adcode', 'model', 'bodyType', 'regYear', 'regMonth']

module = lgb.train(params, lgb_train, num_boost_round=5000, valid_sets=lgb_eval, early_stopping_rounds=100, categorical_feature=categorial_name)

val = module.predict(x_test, num_iteration=module.best_iteration)
val = 2 ** (val - 1)
y_true = 2 ** (y_test.reshape(1, -1)[0] - 1)
nrmse = metrics(y_true, val, val_model.reshape(1, -1)[0])

iters = module.best_iteration + 100
train_all = np.vstack((x_train, x_test))
label_all = np.hstack((y_train, y_test))
lgb_data = lgb.Dataset(train_all, label_all.ravel())
model = lgb.train(params, lgb_data, num_boost_round=iters, categorical_feature=categorial_name)

predict = model.predict(test_feature)
predict = 2 ** (predict - 1)
print(predict)

print('model train over, rmse:', nrmse)   
submit['forecastVolum'] = predict
test_prob_collection = pd.concat([test_prob_collection, submit], axis=0, ignore_index=True)

[1]	valid_0's rmse: 1.7418
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 1.6969
[3]	valid_0's rmse: 1.65345
[4]	valid_0's rmse: 1.61178
[5]	valid_0's rmse: 1.57121
[6]	valid_0's rmse: 1.53179
[7]	valid_0's rmse: 1.49326
[8]	valid_0's rmse: 1.45858
[9]	valid_0's rmse: 1.4225
[10]	valid_0's rmse: 1.38822
[11]	valid_0's rmse: 1.35904
[12]	valid_0's rmse: 1.32697
[13]	valid_0's rmse: 1.29754
[14]	valid_0's rmse: 1.26702
[15]	valid_0's rmse: 1.23918
[16]	valid_0's rmse: 1.21255
[17]	valid_0's rmse: 1.18589
[18]	valid_0's rmse: 1.16155
[19]	valid_0's rmse: 1.13627
[20]	valid_0's rmse: 1.11101
[21]	valid_0's rmse: 1.08871
[22]	valid_0's rmse: 1.06598
[23]	valid_0's rmse: 1.04345
[24]	valid_0's rmse: 1.02099
[25]	valid_0's rmse: 1.00088
[26]	valid_0's rmse: 0.979864
[27]	valid_0's rmse: 0.963174
[28]	valid_0's rmse: 0.942192
[29]	valid_0's rmse: 0.926322
[30]	valid_0's rmse: 0.907426
[31]	valid_0's rmse: 0.887486
[32]	valid_0's rmse: 0.87085
[33]	valid_0's 

[280]	valid_0's rmse: 0.473079
[281]	valid_0's rmse: 0.473058
[282]	valid_0's rmse: 0.472889
[283]	valid_0's rmse: 0.472938
[284]	valid_0's rmse: 0.47289
[285]	valid_0's rmse: 0.472981
[286]	valid_0's rmse: 0.472892
[287]	valid_0's rmse: 0.472911
[288]	valid_0's rmse: 0.472852
[289]	valid_0's rmse: 0.472883
[290]	valid_0's rmse: 0.472889
[291]	valid_0's rmse: 0.472927
[292]	valid_0's rmse: 0.472413
[293]	valid_0's rmse: 0.472629
[294]	valid_0's rmse: 0.472197
[295]	valid_0's rmse: 0.471749
[296]	valid_0's rmse: 0.471677
[297]	valid_0's rmse: 0.47161
[298]	valid_0's rmse: 0.471591
[299]	valid_0's rmse: 0.471569
[300]	valid_0's rmse: 0.471548
[301]	valid_0's rmse: 0.471273
[302]	valid_0's rmse: 0.471307
[303]	valid_0's rmse: 0.471258
[304]	valid_0's rmse: 0.471411
[305]	valid_0's rmse: 0.471506
[306]	valid_0's rmse: 0.471547
[307]	valid_0's rmse: 0.470912
[308]	valid_0's rmse: 0.470814
[309]	valid_0's rmse: 0.470511
[310]	valid_0's rmse: 0.470149
[311]	valid_0's rmse: 0.470164
[312]	vali

```
model train over, rmse: 0.638824675082049
```

### Model-LightGBM - 三月

In [14]:
size, pre = 3, 8   # 5
train_feature  = get_train_feature(size, pre)
test_feature  = get_test_feature(size, pre)

cols = ['province', 'adcode', 'model', 'regYear', 'regMonth', 'bodyType']   #  , 'salesVolume'

temp_train = get_basic_feature(size, pre, 'train')
train_feature = train_feature.drop(drop_cols, axis=1).merge(temp_train, on=cols, how='left')
train_feature

temp_test = get_basic_feature(size, pre, 'test')
test_feature = test_feature.drop(drop_cols, axis=1).merge(temp_test, on=cols, how='left')
test_feature
train_feature.isnull().sum()
test_feature

submit = test_feature[['province', 'adcode', 'model']]
submit['regYear'] = 2018
submit['regMonth'] = 3
##############################

test_index = list(train_feature[(train_feature['regYear'] == 2017) & (train_feature['regMonth'] == 9)].index)

def drop_duplicate(n):
    return n not in test_index

train_index = list(filter(drop_duplicate, list(range(len(train_feature)))))

train_model = train_feature['model'].values[train_index]   # model
val_model = train_feature['model'].values[test_index]

model_set = dict()
for index in range(len(cars)):
    model_set[cars[index]] = index
train_feature['bodyType'] = train_feature['bodyType'].map({'Hatchback': 0, 'MPV': 1, 'SUV': 2, 'Sedan': 3})
train_feature['model'] = train_feature['model'].map(model_set)
test_feature['bodyType'] = test_feature['bodyType'].map({'Hatchback': 0, 'MPV': 1, 'SUV': 2, 'Sedan': 3})
test_feature['model'] = test_feature['model'].map(model_set)

train_label = train_feature[['label']]
train_feature.drop(['province', 'label'], axis=1, inplace=True)
test_feature.drop(['province', 'label'], axis=1, inplace=True)

train_label['log'] = train_label['label'].apply(lambda index: np.log2(index) + 1)
x_train = train_feature.values[train_index]
y_train = train_label['log'].values[train_index]
x_test = train_feature.values[test_index]
y_test = train_label['log'].values[test_index]

     province  adcode             model   bodyType  regYear  regMonth  \
0          浙江  330000  8cbac8dd8f0e89e2        SUV     2016         9   
1          浙江  330000  8cbac8dd8f0e89e2        SUV     2016        10   
2          浙江  330000  8cbac8dd8f0e89e2        SUV     2016        11   
3          浙江  330000  8cbac8dd8f0e89e2        SUV     2016        12   
4          浙江  330000  8cbac8dd8f0e89e2        SUV     2017         1   
...       ...     ...               ...        ...      ...       ...   
6287       河南  410000  5f727f32393ade77  Hatchback     2017         5   
6288       河南  410000  5f727f32393ade77  Hatchback     2017         6   
6289       河南  410000  5f727f32393ade77  Hatchback     2017         7   
6290       河南  410000  5f727f32393ade77  Hatchback     2017         8   
6291       河南  410000  5f727f32393ade77  Hatchback     2017         9   

      salesVolume  is_pring_festival  distance_spring_festival  salesVolume_1  \
0        4.663439                  0      

,province,adcode,model,bodyType,regYear,regMonth,label,salesVolume,is_pring_festival,distance_spring_festival,...,salesVolume_qoq_6,salesVolume_7,salesVolume_diff_7,salesVolume_qoq_7,salesVolume_diff2_1,salesVolume_diff2_2,salesVolume_diff2_3,salesVolume_diff2_4,salesVolume_diff2_5,salesVolume_diff2_6
0,浙江,330000,8cbac8dd8f0e89e2,SUV,2016,9,193.0,4.663439,0,4,...,1.012653,4.204693,0.458746,1.109103,-0.249314,0.112857,0.220169,-0.074742,0.222672,-0.352159
1,浙江,330000,8cbac8dd8f0e89e2,SUV,2016,10,93.0,4.852030,0,3,...,1.095065,4.605170,0.246860,1.053605,0.140273,-0.109041,0.253130,0.360441,0.065531,0.362945
2,浙江,330000,8cbac8dd8f0e89e2,SUV,2016,11,59.0,4.787492,0,2,...,1.051300,4.430817,0.356675,1.080499,-0.253130,-0.112857,-0.362171,0.000000,0.107312,-0.187599
3,浙江,330000,8cbac8dd8f0e89e2,SUV,2016,12,72.0,5.262690,0,1,...,1.200972,4.553877,0.708813,1.155651,0.539737,0.286607,0.426880,0.177566,0.539737,0.647049
4,浙江,330000,8cbac8dd8f0e89e2,SUV,2017,1,49.0,4.532599,1,0,...,1.049823,4.382027,0.150573,1.034361,-1.205289,-0.665552,-0.918682,-0.778409,-1.027723,-0.665552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6287,河南,410000,5f727f32393ade77,Hatchback,2017,5,323.0,5.945421,0,4,...,0.981994,5.894403,0.051018,1.008655,-0.152733,-0.493669,-0.473157,0.685095,-0.426131,-0.321639
6288,河南,410000,5f727f32393ade77,Hatchback,2017,6,370.0,6.018593,0,5,...,0.952465,6.054439,-0.035846,0.994079,0.234775,0.082042,-0.258894,-0.238382,0.919870,-0.191356
6289,河南,410000,5f727f32393ade77,Hatchback,2017,7,309.0,6.023448,0,6,...,1.100722,6.318968,-0.295521,0.953233,-0.068318,0.166457,0.013724,-0.327213,-0.306700,0.851552
6290,河南,410000,5f727f32393ade77,Hatchback,2017,8,367.0,5.777652,0,6,...,0.998933,5.472271,0.305382,1.055805,-0.250650,-0.318968,-0.084193,-0.236926,-0.577862,-0.557350


    province  adcode             model   bodyType  regYear  regMonth  \
0         浙江  330000  8cbac8dd8f0e89e2        SUV     2017        12   
1         福建  350000  8cbac8dd8f0e89e2        SUV     2017        12   
2         四川  510000  8cbac8dd8f0e89e2        SUV     2017        12   
3         陕西  610000  8cbac8dd8f0e89e2        SUV     2017        12   
4         安徽  340000  8cbac8dd8f0e89e2        SUV     2017        12   
..       ...     ...               ...        ...      ...       ...   
479       重庆  500000  5f727f32393ade77  Hatchback     2017        12   
480       河北  130000  5f727f32393ade77  Hatchback     2017        12   
481       山西  140000  5f727f32393ade77  Hatchback     2017        12   
482       江西  360000  5f727f32393ade77  Hatchback     2017        12   
483       河南  410000  5f727f32393ade77  Hatchback     2017        12   

     salesVolume  is_pring_festival  distance_spring_festival  salesVolume_1  \
0       4.248495                  0                    

,province,adcode,model,bodyType,regYear,regMonth,label,salesVolume,is_pring_festival,distance_spring_festival,...,salesVolume_qoq_6,salesVolume_7,salesVolume_diff_7,salesVolume_qoq_7,salesVolume_diff2_1,salesVolume_diff2_2,salesVolume_diff2_3,salesVolume_diff2_4,salesVolume_diff2_5,salesVolume_diff2_6
0,浙江,330000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,4.248495,0,2,...,1.070071,4.248495,0.000000,1.000000,1.554908,0.902801,0.970669,0.473205,0.707263,1.092711
1,福建,350000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,4.204693,0,2,...,0.962294,4.828314,-0.623621,0.870841,-0.677515,-0.568700,-0.589720,-0.292964,-0.999731,-0.090032
2,四川,510000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,4.941642,0,2,...,1.092851,4.382027,0.559616,1.127707,0.596242,0.399192,0.292823,0.546400,-0.052922,0.227170
3,陕西,610000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,5.438079,0,2,...,1.317640,4.110874,1.327205,1.322852,1.560052,0.867843,0.651712,1.219107,0.433917,0.991002
4,安徽,340000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,3.258097,0,2,...,1.175110,3.583519,-0.325422,0.909189,0.238411,-0.277632,-0.820981,0.241162,0.133531,0.810930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,重庆,500000,5f727f32393ade77,Hatchback,2017,12,NaN,4.852030,0,2,...,1.051333,4.418841,0.433190,1.098032,0.251104,0.551030,0.061252,0.401701,0.479454,0.133962
480,河北,130000,5f727f32393ade77,Hatchback,2017,12,NaN,6.991177,0,2,...,1.038835,6.895683,0.095494,1.013848,0.413524,0.632738,0.632097,0.575226,0.533479,0.673928
481,山西,140000,5f727f32393ade77,Hatchback,2017,12,NaN,5.863631,0,2,...,1.072342,5.337538,0.526093,1.098565,0.589077,0.601244,0.594485,0.524223,0.597950,0.419903
482,江西,360000,5f727f32393ade77,Hatchback,2017,12,NaN,5.455321,0,2,...,1.105553,5.036953,0.418369,1.083060,0.100234,0.312397,0.419217,0.119323,0.203010,0.381650


province                     0
adcode                       0
model                        0
bodyType                     0
regYear                      0
regMonth                     0
label                        0
salesVolume                  0
is_pring_festival            0
distance_spring_festival     0
salesVolume_1                0
salesVolume_diff_1           0
salesVolume_qoq_1           10
salesVolume_2                0
salesVolume_diff_2           0
salesVolume_qoq_2            9
salesVolume_3                0
salesVolume_diff_3           0
salesVolume_qoq_3            9
salesVolume_4                0
salesVolume_diff_4           0
salesVolume_qoq_4            8
salesVolume_5                0
salesVolume_diff_5           0
salesVolume_qoq_5            9
salesVolume_6                0
salesVolume_diff_6           0
salesVolume_qoq_6            7
salesVolume_7                0
salesVolume_diff_7           0
salesVolume_qoq_7            7
salesVolume_diff2_1          0
salesVol

,province,adcode,model,bodyType,regYear,regMonth,label,salesVolume,is_pring_festival,distance_spring_festival,...,salesVolume_qoq_6,salesVolume_7,salesVolume_diff_7,salesVolume_qoq_7,salesVolume_diff2_1,salesVolume_diff2_2,salesVolume_diff2_3,salesVolume_diff2_4,salesVolume_diff2_5,salesVolume_diff2_6
0,浙江,330000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,4.248495,0,2,...,1.070071,4.248495,0.000000,1.000000,1.554908,0.902801,0.970669,0.473205,0.707263,1.092711
1,福建,350000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,4.204693,0,2,...,0.962294,4.828314,-0.623621,0.870841,-0.677515,-0.568700,-0.589720,-0.292964,-0.999731,-0.090032
2,四川,510000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,4.941642,0,2,...,1.092851,4.382027,0.559616,1.127707,0.596242,0.399192,0.292823,0.546400,-0.052922,0.227170
3,陕西,610000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,5.438079,0,2,...,1.317640,4.110874,1.327205,1.322852,1.560052,0.867843,0.651712,1.219107,0.433917,0.991002
4,安徽,340000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,3.258097,0,2,...,1.175110,3.583519,-0.325422,0.909189,0.238411,-0.277632,-0.820981,0.241162,0.133531,0.810930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,重庆,500000,5f727f32393ade77,Hatchback,2017,12,NaN,4.852030,0,2,...,1.051333,4.418841,0.433190,1.098032,0.251104,0.551030,0.061252,0.401701,0.479454,0.133962
480,河北,130000,5f727f32393ade77,Hatchback,2017,12,NaN,6.991177,0,2,...,1.038835,6.895683,0.095494,1.013848,0.413524,0.632738,0.632097,0.575226,0.533479,0.673928
481,山西,140000,5f727f32393ade77,Hatchback,2017,12,NaN,5.863631,0,2,...,1.072342,5.337538,0.526093,1.098565,0.589077,0.601244,0.594485,0.524223,0.597950,0.419903
482,江西,360000,5f727f32393ade77,Hatchback,2017,12,NaN,5.455321,0,2,...,1.105553,5.036953,0.418369,1.083060,0.100234,0.312397,0.419217,0.119323,0.203010,0.381650


In [15]:
# LightGBM model
params = {
      'boosting_type': 'gbdt',
      'objective': 'rmse',
      'metric': ['rmse'],   # 'l2', 'binary_logloss',
      'learning_rate': 0.03,
      'num_leaves': 2 ** 5 - 1,
      # 'min_child_samples': 100,
      'max_depth': 6,
      'subsample': 0.8,
      'subsample_freq': 5,
      'colsample_bytree': 0.8,
      'seed': 2019,
      'nthread': -1,
      'verbose': 1,
}

lgb_train = lgb.Dataset(x_train, y_train.ravel())
lgb_eval = lgb.Dataset(x_test, y_test.ravel(), reference=lgb_train)
# categorial_name = ['adcode', 'model', 'bodyType', 'regYear', 'regMonth']

module = lgb.train(params, lgb_train, num_boost_round=5000, valid_sets=lgb_eval, early_stopping_rounds=100, categorical_feature=categorial_name)

val = module.predict(x_test, num_iteration=module.best_iteration)
val = 2 ** (val - 1)
y_true = 2 ** (y_test.reshape(1, -1)[0] - 1)
nrmse = metrics(y_true, val, val_model.reshape(1, -1)[0])

iters = module.best_iteration + 100
train_all = np.vstack((x_train, x_test))
label_all = np.hstack((y_train, y_test))
lgb_data = lgb.Dataset(train_all, label_all.ravel())
model = lgb.train(params, lgb_data, num_boost_round=iters, categorical_feature=categorial_name)

predict = model.predict(test_feature)
predict = 2 ** (predict - 1)
print(predict)

print('model train over, rmse:', nrmse)
submit['forecastVolum'] = predict
test_prob_collection = pd.concat([test_prob_collection, submit], axis=0, ignore_index=True)

[1]	valid_0's rmse: 1.74065
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 1.69612
[3]	valid_0's rmse: 1.65409
[4]	valid_0's rmse: 1.61314
[5]	valid_0's rmse: 1.5725
[6]	valid_0's rmse: 1.53461
[7]	valid_0's rmse: 1.4983
[8]	valid_0's rmse: 1.46168
[9]	valid_0's rmse: 1.42681
[10]	valid_0's rmse: 1.3931
[11]	valid_0's rmse: 1.36292
[12]	valid_0's rmse: 1.33046
[13]	valid_0's rmse: 1.30029
[14]	valid_0's rmse: 1.2723
[15]	valid_0's rmse: 1.24363
[16]	valid_0's rmse: 1.21491
[17]	valid_0's rmse: 1.1881
[18]	valid_0's rmse: 1.16179
[19]	valid_0's rmse: 1.1357
[20]	valid_0's rmse: 1.11222
[21]	valid_0's rmse: 1.08844
[22]	valid_0's rmse: 1.06499
[23]	valid_0's rmse: 1.04399
[24]	valid_0's rmse: 1.0229
[25]	valid_0's rmse: 1.00428
[26]	valid_0's rmse: 0.983775
[27]	valid_0's rmse: 0.964393
[28]	valid_0's rmse: 0.94477
[29]	valid_0's rmse: 0.927956
[30]	valid_0's rmse: 0.909813
[31]	valid_0's rmse: 0.890708
[32]	valid_0's rmse: 0.874857
[33]	valid_0's rmse

```
model train over, rmse: 0.5984062496994993
```

### Model-LightGBM - 四月

In [16]:
size, pre = 4, 7   # 5
train_feature  = get_train_feature(size, pre)
test_feature  = get_test_feature(size, pre)

cols = ['province', 'adcode', 'model', 'regYear', 'regMonth', 'bodyType']   #  , 'salesVolume'

temp_train = get_basic_feature(size, pre, 'train')
train_feature = train_feature.drop(drop_cols, axis=1).merge(temp_train, on=cols, how='left')
train_feature

temp_test = get_basic_feature(size, pre, 'test')
test_feature = test_feature.drop(drop_cols, axis=1).merge(temp_test, on=cols, how='left')
test_feature
train_feature.isnull().sum()
test_feature


submit = test_feature[['province', 'adcode', 'model']]
submit['regYear'] = 2018
submit['regMonth'] = 4
###############################

test_index = list(train_feature[(train_feature['regYear'] == 2017) & (train_feature['regMonth'] == 8)].index)

def drop_duplicate(n):
    return n not in test_index

train_index = list(filter(drop_duplicate, list(range(len(train_feature)))))

train_model = train_feature['model'].values[train_index]   # model
val_model = train_feature['model'].values[test_index]

model_set = dict()
for index in range(len(cars)):
    model_set[cars[index]] = index
train_feature['bodyType'] = train_feature['bodyType'].map({'Hatchback': 0, 'MPV': 1, 'SUV': 2, 'Sedan': 3})
train_feature['model'] = train_feature['model'].map(model_set)
test_feature['bodyType'] = test_feature['bodyType'].map({'Hatchback': 0, 'MPV': 1, 'SUV': 2, 'Sedan': 3})
test_feature['model'] = test_feature['model'].map(model_set)

train_label = train_feature[['label']]
train_feature.drop(['province', 'label'], axis=1, inplace=True)
test_feature.drop(['province', 'label'], axis=1, inplace=True)

train_label['log'] = train_label['label'].apply(lambda index: np.log2(index) + 1)
x_train = train_feature.values[train_index]
y_train = train_label['log'].values[train_index]
x_test = train_feature.values[test_index]
y_test = train_label['log'].values[test_index]

     province  adcode             model   bodyType  regYear  regMonth  \
0          浙江  330000  8cbac8dd8f0e89e2        SUV     2016         8   
1          浙江  330000  8cbac8dd8f0e89e2        SUV     2016         9   
2          浙江  330000  8cbac8dd8f0e89e2        SUV     2016        10   
3          浙江  330000  8cbac8dd8f0e89e2        SUV     2016        11   
4          浙江  330000  8cbac8dd8f0e89e2        SUV     2016        12   
...       ...     ...               ...        ...      ...       ...   
6287       河南  410000  5f727f32393ade77  Hatchback     2017         4   
6288       河南  410000  5f727f32393ade77  Hatchback     2017         5   
6289       河南  410000  5f727f32393ade77  Hatchback     2017         6   
6290       河南  410000  5f727f32393ade77  Hatchback     2017         7   
6291       河南  410000  5f727f32393ade77  Hatchback     2017         8   

      salesVolume  is_pring_festival  distance_spring_festival  salesVolume_1  \
0        4.615121                  0      

,province,adcode,model,bodyType,regYear,regMonth,label,salesVolume,is_pring_festival,distance_spring_festival,...,salesVolume_diff_5,salesVolume_qoq_5,salesVolume_6,salesVolume_diff_6,salesVolume_qoq_6,salesVolume_diff2_1,salesVolume_diff2_2,salesVolume_diff2_3,salesVolume_diff2_4,salesVolume_diff2_5
0,浙江,330000,8cbac8dd8f0e89e2,SUV,2016,8,193.0,4.615121,0,5,...,0.009950,1.002161,4.204693,0.410428,1.097612,0.362171,0.469483,0.174572,0.471986,-0.102845
1,浙江,330000,8cbac8dd8f0e89e2,SUV,2016,9,93.0,4.663439,0,4,...,0.232622,1.052501,4.605170,0.058269,1.012653,-0.249314,0.112857,0.220169,-0.074742,0.222672
2,浙江,330000,8cbac8dd8f0e89e2,SUV,2016,10,59.0,4.852030,0,3,...,0.298153,1.065472,4.430817,0.421213,1.095065,0.140273,-0.109041,0.253130,0.360441,0.065531
3,浙江,330000,8cbac8dd8f0e89e2,SUV,2016,11,72.0,4.787492,0,2,...,0.405465,1.092529,4.553877,0.233615,1.051300,-0.253130,-0.112857,-0.362171,0.000000,0.107312
4,浙江,330000,8cbac8dd8f0e89e2,SUV,2016,12,49.0,5.262690,0,1,...,0.945202,1.218924,4.382027,0.880664,1.200972,0.539737,0.286607,0.426880,0.177566,0.539737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6287,河南,410000,5f727f32393ade77,Hatchback,2017,4,323.0,6.107023,0,3,...,0.052584,1.008685,5.894403,0.212620,1.036072,-0.340936,-0.320424,0.837828,-0.273398,-0.168906
6288,河南,410000,5f727f32393ade77,Hatchback,2017,5,370.0,5.945421,0,4,...,-0.373548,0.940885,6.054439,-0.109019,0.981994,-0.152733,-0.493669,-0.473157,0.685095,-0.426131
6289,河南,410000,5f727f32393ade77,Hatchback,2017,6,309.0,6.018593,0,5,...,0.546323,1.099835,6.318968,-0.300375,0.952465,0.234775,0.082042,-0.258894,-0.238382,0.919870
6290,河南,410000,5f727f32393ade77,Hatchback,2017,7,367.0,6.023448,0,6,...,0.239622,1.041430,5.472271,0.551177,1.100722,-0.068318,0.166457,0.013724,-0.327213,-0.306700


    province  adcode             model   bodyType  regYear  regMonth  \
0         浙江  330000  8cbac8dd8f0e89e2        SUV     2017        12   
1         福建  350000  8cbac8dd8f0e89e2        SUV     2017        12   
2         四川  510000  8cbac8dd8f0e89e2        SUV     2017        12   
3         陕西  610000  8cbac8dd8f0e89e2        SUV     2017        12   
4         安徽  340000  8cbac8dd8f0e89e2        SUV     2017        12   
..       ...     ...               ...        ...      ...       ...   
479       重庆  500000  5f727f32393ade77  Hatchback     2017        12   
480       河北  130000  5f727f32393ade77  Hatchback     2017        12   
481       山西  140000  5f727f32393ade77  Hatchback     2017        12   
482       江西  360000  5f727f32393ade77  Hatchback     2017        12   
483       河南  410000  5f727f32393ade77  Hatchback     2017        12   

     salesVolume  is_pring_festival  distance_spring_festival  salesVolume_1  \
0       4.248495                  0                    

,province,adcode,model,bodyType,regYear,regMonth,label,salesVolume,is_pring_festival,distance_spring_festival,...,salesVolume_diff_5,salesVolume_qoq_5,salesVolume_6,salesVolume_diff_6,salesVolume_qoq_6,salesVolume_diff2_1,salesVolume_diff2_2,salesVolume_diff2_3,salesVolume_diff2_4,salesVolume_diff2_5
0,浙江,330000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,4.248495,0,2,...,0.170958,1.041927,3.970292,0.278203,1.070071,1.554908,0.902801,0.970669,0.473205,0.707263
1,福建,350000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,4.204693,0,2,...,-0.615589,0.872292,4.369448,-0.164755,0.962294,-0.677515,-0.568700,-0.589720,-0.292964,-0.999731
2,四川,510000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,4.941642,0,2,...,0.000000,1.000000,4.521789,0.419854,1.092851,0.596242,0.399192,0.292823,0.546400,-0.052922
3,陕西,610000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,5.438079,0,2,...,0.737599,1.156920,4.127134,1.310945,1.317640,1.560052,0.867843,0.651712,1.219107,0.433917
4,安徽,340000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,3.258097,0,2,...,0.619039,1.234568,2.772589,0.485508,1.175110,0.238411,-0.277632,-0.820981,0.241162,0.133531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,重庆,500000,5f727f32393ade77,Hatchback,2017,12,NaN,4.852030,0,2,...,0.386122,1.086460,4.615121,0.236910,1.051333,0.251104,0.551030,0.061252,0.401701,0.479454
480,河北,130000,5f727f32393ade77,Hatchback,2017,12,NaN,6.991177,0,2,...,0.286763,1.042772,6.729824,0.261353,1.038835,0.413524,0.632738,0.632097,0.575226,0.533479
481,山西,140000,5f727f32393ade77,Hatchback,2017,12,NaN,5.863631,0,2,...,0.443096,1.081744,5.468060,0.395571,1.072342,0.589077,0.601244,0.594485,0.524223,0.597950
482,江西,360000,5f727f32393ade77,Hatchback,2017,12,NaN,5.455321,0,2,...,0.444686,1.088748,4.934474,0.520847,1.105553,0.100234,0.312397,0.419217,0.119323,0.203010


province                     0
adcode                       0
model                        0
bodyType                     0
regYear                      0
regMonth                     0
label                        0
salesVolume                  0
is_pring_festival            0
distance_spring_festival     0
salesVolume_1                0
salesVolume_diff_1           0
salesVolume_qoq_1           10
salesVolume_2                0
salesVolume_diff_2           0
salesVolume_qoq_2            9
salesVolume_3                0
salesVolume_diff_3           0
salesVolume_qoq_3            8
salesVolume_4                0
salesVolume_diff_4           0
salesVolume_qoq_4            8
salesVolume_5                0
salesVolume_diff_5           0
salesVolume_qoq_5            8
salesVolume_6                0
salesVolume_diff_6           0
salesVolume_qoq_6            6
salesVolume_diff2_1          0
salesVolume_diff2_2          0
salesVolume_diff2_3          0
salesVolume_diff2_4          0
salesVol

,province,adcode,model,bodyType,regYear,regMonth,label,salesVolume,is_pring_festival,distance_spring_festival,...,salesVolume_diff_5,salesVolume_qoq_5,salesVolume_6,salesVolume_diff_6,salesVolume_qoq_6,salesVolume_diff2_1,salesVolume_diff2_2,salesVolume_diff2_3,salesVolume_diff2_4,salesVolume_diff2_5
0,浙江,330000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,4.248495,0,2,...,0.170958,1.041927,3.970292,0.278203,1.070071,1.554908,0.902801,0.970669,0.473205,0.707263
1,福建,350000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,4.204693,0,2,...,-0.615589,0.872292,4.369448,-0.164755,0.962294,-0.677515,-0.568700,-0.589720,-0.292964,-0.999731
2,四川,510000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,4.941642,0,2,...,0.000000,1.000000,4.521789,0.419854,1.092851,0.596242,0.399192,0.292823,0.546400,-0.052922
3,陕西,610000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,5.438079,0,2,...,0.737599,1.156920,4.127134,1.310945,1.317640,1.560052,0.867843,0.651712,1.219107,0.433917
4,安徽,340000,8cbac8dd8f0e89e2,SUV,2017,12,NaN,3.258097,0,2,...,0.619039,1.234568,2.772589,0.485508,1.175110,0.238411,-0.277632,-0.820981,0.241162,0.133531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,重庆,500000,5f727f32393ade77,Hatchback,2017,12,NaN,4.852030,0,2,...,0.386122,1.086460,4.615121,0.236910,1.051333,0.251104,0.551030,0.061252,0.401701,0.479454
480,河北,130000,5f727f32393ade77,Hatchback,2017,12,NaN,6.991177,0,2,...,0.286763,1.042772,6.729824,0.261353,1.038835,0.413524,0.632738,0.632097,0.575226,0.533479
481,山西,140000,5f727f32393ade77,Hatchback,2017,12,NaN,5.863631,0,2,...,0.443096,1.081744,5.468060,0.395571,1.072342,0.589077,0.601244,0.594485,0.524223,0.597950
482,江西,360000,5f727f32393ade77,Hatchback,2017,12,NaN,5.455321,0,2,...,0.444686,1.088748,4.934474,0.520847,1.105553,0.100234,0.312397,0.419217,0.119323,0.203010


In [17]:
# LightGBM model
params = {
      'boosting_type': 'gbdt',
      'objective': 'rmse',
      'metric': ['rmse'],   # 'l2', 'binary_logloss',
      'learning_rate': 0.03,
      'num_leaves': 2 ** 5 - 1,
      # 'min_child_samples': 100,
      'max_depth': 6,
      'subsample': 0.8,
      'subsample_freq': 5,
      'colsample_bytree': 0.8,
      'seed': 2019,
      'nthread': -1,
      'verbose': 1,
}

lgb_train = lgb.Dataset(x_train, y_train.ravel())
lgb_eval = lgb.Dataset(x_test, y_test.ravel(), reference=lgb_train)
# categorial_name = ['adcode', 'model', 'bodyType', 'regYear', 'regMonth']

module = lgb.train(params, lgb_train, num_boost_round=5000, valid_sets=lgb_eval, early_stopping_rounds=100, categorical_feature=categorial_name)

val = module.predict(x_test, num_iteration=module.best_iteration)
val = 2 ** (val - 1)
y_true = 2 ** (y_test.reshape(1, -1)[0] - 1)
nrmse = metrics(y_true, val, val_model.reshape(1, -1)[0])

iters = module.best_iteration + 100
train_all = np.vstack((x_train, x_test))
label_all = np.hstack((y_train, y_test))
lgb_data = lgb.Dataset(train_all, label_all.ravel())
model = lgb.train(params, lgb_data, num_boost_round=iters, categorical_feature=categorial_name)

predict = model.predict(test_feature)
predict = 2 ** (predict - 1)
print(predict)

print('model train over, rmse:', nrmse)
submit['forecastVolum'] = predict
test_prob_collection = pd.concat([test_prob_collection, submit], axis=0, ignore_index=True)

[1]	valid_0's rmse: 1.74265
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 1.70432
[3]	valid_0's rmse: 1.66379
[4]	valid_0's rmse: 1.62416
[5]	valid_0's rmse: 1.58925
[6]	valid_0's rmse: 1.55074
[7]	valid_0's rmse: 1.51459
[8]	valid_0's rmse: 1.47955
[9]	valid_0's rmse: 1.44875
[10]	valid_0's rmse: 1.41511
[11]	valid_0's rmse: 1.38264
[12]	valid_0's rmse: 1.35454
[13]	valid_0's rmse: 1.32509
[14]	valid_0's rmse: 1.29835
[15]	valid_0's rmse: 1.27295
[16]	valid_0's rmse: 1.2505
[17]	valid_0's rmse: 1.22294
[18]	valid_0's rmse: 1.19739
[19]	valid_0's rmse: 1.17249
[20]	valid_0's rmse: 1.14917
[21]	valid_0's rmse: 1.12886
[22]	valid_0's rmse: 1.1055
[23]	valid_0's rmse: 1.08352
[24]	valid_0's rmse: 1.06418
[25]	valid_0's rmse: 1.04355
[26]	valid_0's rmse: 1.02363
[27]	valid_0's rmse: 1.00207
[28]	valid_0's rmse: 0.982909
[29]	valid_0's rmse: 0.965963
[30]	valid_0's rmse: 0.946678
[31]	valid_0's rmse: 0.926634
[32]	valid_0's rmse: 0.90751
[33]	valid_0's r

```
model train over, rmse: 0.571379011888757
```

In [18]:
train_feature.shape

(6292, 31)

In [19]:
test_prob_collection

,province,adcode,model,regYear,regMonth,forecastVolum
0,浙江,330000,8cbac8dd8f0e89e2,2018,1,57.150108
1,福建,350000,8cbac8dd8f0e89e2,2018,1,65.454914
2,四川,510000,8cbac8dd8f0e89e2,2018,1,119.002727
3,陕西,610000,8cbac8dd8f0e89e2,2018,1,160.121146
4,安徽,340000,8cbac8dd8f0e89e2,2018,1,27.833133
...,...,...,...,...,...,...
1931,重庆,500000,5f727f32393ade77,2018,4,95.119093
1932,河北,130000,5f727f32393ade77,2018,4,709.999948
1933,山西,140000,5f727f32393ade77,2018,4,225.299831
1934,江西,360000,5f727f32393ade77,2018,4,150.864040


In [20]:
test_prob_collection.index = range(len(test_prob_collection))
evaluation_public = evaluation_public.drop('forecastVolum', axis=1).merge(test_prob_collection, on=['province', 'adcode', 'model', 'regYear', 'regMonth'], how='left')
evaluation_public['forecastVolum'] = evaluation_public['forecastVolum'].apply(lambda index: int(np.round(index)))
evaluation_public['forecastVolum'] = evaluation_public.apply(lambda index: index.forecastVolum / 1.15 if index.regMonth in [1, 2] else index.forecastVolum / 1.45, axis=1)
evaluation_public['forecastVolum'] = evaluation_public['forecastVolum'].apply(lambda index: int(np.round(index)))
evaluation_public['forecastVolum'].mean()

177.31714876033058

In [21]:
sub = evaluation_public.copy()
sub.groupby(['regMonth'], as_index=False)['forecastVolum'].mean()
sub['forecastVolum'].mean()
sub[['id', 'forecastVolum']].to_csv('./submit/sub_lj.csv', encoding='utf-8', index=None)

,regMonth,forecastVolum
0,1,232.064050
1,2,162.758264
2,3,158.088843
3,4,156.357438


177.31714876033058